In [ ]:
# 必要ライブラリの導入
!pip install japanize-matplotlib | tail -n 1

# 必要ライブラリのimport
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 日本語化対応
import japanize_matplotlib #日本語化matplotlib

# 営業成約予測モデル

以下のガイドに従って、**Watson Machine Laring認証情報**と、**Scoring URL**を調べて、結果を該当セルにコピーしてください。

### Watson Machine Learning認証情報の設定

##### 認証情報の調べ方  

(1) Watson Studioの**Settings**タブの**Associated services**の欄を見て、Machine Learningサービスの名称を調べます。  
![](https://github.com/makaishi2/sample-data/raw/master/images/waston-ml-01.png)
画面例の場合は、`pm-20-ht`。

(2) IBM Cloud管理画面 https://cloud.ibm.com/resources で、サービス名で検索をして、Machine Learningサービスを表示させます。
 ![](https://github.com/makaishi2/sample-data/raw/master/images/watson-ml-02.png)
    
(3) サービスのリンクをクリックします。

(4) **サービス資格情報** -> **資格情報の表示** をクリックして、資格情報をクリップボードにコピーします。
 ![](https://github.com/makaishi2/sample-data/raw/master/images/watson-ml-03.png)
    
(5) コピーした結果を下のセルに張り付けます。  
(必要なのは`instance_id`,`apikey`,`url`のみですが、他の項目が残っていても構いません。)

In [ ]:
wml_credentials = {
                   "url": "https://us-south.ml.cloud.ibm.com",
                   "apikey":"xxxx",
                   "instance_id": "xxxx"
                  }


### APIのScoring URL設定

#### Scoring URLの調べ方

(1) Watson Studioのプロジェクト管理画面から**デプロイメント**タブを選択  
リストの中から**営業成約予測WEB**のリンクをクリック

 ![](https://github.com/makaishi2/sample-data/raw/master/images/watson-ml-04.png)

(2) デプロイメントの詳細画面で**実装**タブをクリックし、**エンドポイントのスコアリング**をコピーします。
    
 ![](https://github.com/makaishi2/sample-data/raw/master/images/watson-ml-05.png)    

In [ ]:
# scoring URLの設定
scoring_url = "xxxx"

### 必要ライブラリのimport

In [ ]:
import urllib3, requests, json
import urllib.request

### Watson ML呼出し準備

In [ ]:
# トークン取得
apikey = wml_credentials["apikey"]

# Get an IAM token from IBM Cloud
url     = "https://iam.bluemix.net/oidc/token"
headers = { "Content-Type" : "application/x-www-form-urlencoded" }
data    = "apikey=" + apikey + "&grant_type=urn:ibm:params:oauth:grant-type:apikey"
IBM_cloud_IAM_uid = "bx"
IBM_cloud_IAM_pwd = "bx"
response  = requests.post( url, headers=headers, data=data, 
        auth=( IBM_cloud_IAM_uid, IBM_cloud_IAM_pwd ) )
iam_token = response.json()["access_token"]

print('iam_token = ', iam_token)

In [ ]:
# Header の組立て
header_token = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + iam_token}
ml_instance_id = wml_credentials["instance_id"]
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + iam_token, 'ML-Instance-ID': ml_instance_id}

### テスト用CSV読み込み
学習に利用していない 18085件のデータをまとめて読み込みます。

In [ ]:
# ファイルをカレントディレクトリにダウンロード
csv_url = 'https://raw.githubusercontent.com/makaishi2/sample-data/master/data/bank-test-autoai.csv'

In [ ]:
# データフレームに読み込み
df_bank_test = pd.read_csv(csv_url)

In [ ]:
display(df_bank_test.head())
print(df_bank_test.shape)

### 正解データ correctの抽出

In [ ]:
correct = df_bank_test['今回販促結果'].values
correct_b = (correct == 'yes').astype(np.int)
print(correct_b[:50])

In [ ]:
display(df_bank_test['今回販促結果'].value_counts())

In [ ]:
org_rate = df_bank_test['今回販促結果'].value_counts().yes/ len(df_bank_test)
print('%.4f' % org_rate)

### API呼出し用入力データ配列の作成

In [ ]:
df_sub = df_bank_test.copy()

# 目的変数列の削除
df_sub = df_sub.drop('今回販促結果', axis=1)

# np.nan値をNoneに置き換え
df_sub = df_sub.where((pd.notnull(df_sub)), None)

In [ ]:
display(df_sub.head())

In [ ]:
# DataFrameから項目名Listの生成
fields = df_sub.columns.tolist()
print(fields)

In [ ]:
# DataFrameから入力用Listの生成
values = df_sub.values.tolist()
# テスト用
#values = df_sub.head(2).values.tolist()
print(values[:2])

In [ ]:
# payload変数の組立て
payload_scoring = {"input_data": [{
    "fields": fields, 
    "values": values
}]}

### APIの呼出し

In [ ]:
# APIの呼出し
response_scoring = requests.post(scoring_url,  json=payload_scoring, headers=header)

### 戻り値のparese

In [ ]:
# 戻り値のparse
res = json.loads(response_scoring.text)

pred = res['predictions'][0]
pred_values = pred['values']
pred_fields = pred['fields']
df_res = pd.DataFrame(pred_values, columns = pred_fields)

In [ ]:
display(df_res.head())

### 精度評価

In [ ]:
# 予測値の抽出
prediction = df_res['prediction'].values

# 確率値のリスト取得
w1 = df_res['probability'].values.tolist()
probas = [item[1] for item in w1]

In [ ]:
# Accuracyの計算
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(correct, prediction)

In [ ]:
# ROC AUCの計算
from sklearn.metrics import auc
from sklearn.metrics import roc_curve

# fpr, tpr値の取得
fpr, tpr, thresholds = roc_curve(correct, probas, pos_label='yes')

# ROC AUCの計算
roc_auc = auc(fpr, tpr)

In [ ]:
# 混同行列表示用関数

def make_cm(matrix, columns):
    # matrix numpy配列
    
    # columns 項目名リスト
    n = len(columns)
    
    # '正解データ'をn回繰り返すリスト生成
    act = ['正解データ'] * n
    pred = ['予測結果'] * n
    
    #データフレーム生成
    cm = pd.DataFrame(matrix, 
        columns=[pred, columns], index=[act, columns])
    return cm

In [ ]:
# 混同行列の計算
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(correct, prediction)
df_matrix = make_cm(matrix, ['失敗', '成功'])

### 精度評価結果表示

In [ ]:
### 混同行列表示
display(df_matrix)

In [ ]:
# precision / recall
from sklearn.metrics import classification_report
print(classification_report(correct, prediction))

#### 上記の意味

元々は11.7%しか成約がなかったのが、  
モデルで予測することにより**43%成約できる**ことになります。  
また、**成約できそうな顧客の88%はこのモデルで洗い出しができる**ことになります。

In [ ]:
# AccuracyとROC AUCの表示

print("Accuracy: %0.4f" % accuracy)
print("ROC AUC : %0.4f" % roc_auc)

In [ ]:
# ROC曲線の描画

%matplotlib inline 
import matplotlib.pyplot as plt

plt.figure(figsize=(6,6))
plt.clf()
plt.plot([0, 1], [0, 1], 'k--')
plt.fill_between(fpr, tpr, label='ROC Curve')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.title("ROC曲線 AUC値 : %0.3f" % roc_auc)
plt.show()

In [ ]:
# 正解データの yesとnoの数の計算
y_count = np.count_nonzero(correct == 'yes')
n_count = np.count_nonzero(correct == 'no')
print(y_count, n_count)

# 予測値がyesだったもののうち、正解データのyesとnoの数
yy_count = df_matrix.yes.yes
yn_count = df_matrix.yes.no
print(yy_count, yn_count)

In [ ]:
# モデルを使わない場合と使った場合の成約率の比較

plt.subplot(1, 2, 1)
label = ['yes', 'no']
colors = ["lightpink", "lightblue"]
x = np.array([y_count, n_count])
plt.pie(x, labels=label, colors=colors,
       autopct="%1.1f%%", pctdistance=0.7)
plt.title('モデル未導入時', fontsize=14)
plt.axis('equal')
 
plt.subplot(1, 2, 2)
label = ['yes', 'no']
colors = ["lightpink", "lightblue"]
x = np.array([yy_count, yn_count])
plt.pie(x, labels=label, colors=colors,
       autopct="%1.1f%%", pctdistance=0.7)
plt.title('モデル導入後', fontsize=14)
plt.axis('equal')

plt.show()